# Pre-Experiment
Define All Variables to Run the Experiment
    <p>```setup.bash``` Exports the Variables to a ```setup.conf``` and sends it to the server</p>

In [ ]:
username=yousifd # Username of user on server
exp_name=ye # Name of Experiment on NOME
proj_name=cops # Name of Project on NOME
default_node=h0 # Default node to run the experiment from
host_ip=nome.nmc-probe.org
default_ip=${default_node}.${exp_name}.${proj_name}

# Setup Number of Servers and Clients
nservers=8 # Number of Servers and Clients
ndcs=2 # Number of Data Centers
ipprefix=h
ippostfix="-dib"

# Initialize Consistent Variables
trial=1 # Number of Trials in the Experiment
nkeys=100000 # Number of keys for the Experiment
run_time=30 # Time to run the Expierment
trim=5 # Trimmed time from the gathered results

# Initialize Default Variables
value_size=128 # Size of the Value Read and/or Written
cols_per_key_read=5 # Columns per key read
cols_per_key_write=5 # Columns per key write
keys_per_read=5 # Keys per read
keys_per_write=5 # Keys per write
write_frac="0.1" # Fraction of Write Operations to Read Operations
write_trans_frac="0.5" # Fraction of Write Transcation to Read Transactions

# Specify the Experiment's Indepdent Variable and its Values
exp=columns_per_read # Name of the Experiment
indep_values="1:${nkeys} 2:${nkeys}" # Independen Variables to be Experimented with

alias server="ssh ${username}@${host_ip} -o StrictHostKeyChecking=no ssh ${default_ip} -t -t -o StrictHostKeyChecking=no"

source /home/yousifd/setup.bash

# Experiment
Launch Instances of the Distributed System
    
Setup Benchmark on Clients
    
Start the Experiment
    
Kill all Instances of the Distributed System

In [ ]:
server <<- 'EOF'

cd /users/$USER/cops2/experiments/

source dynamic_common

insert_cmd=INSERTCL
KILLALL_SSH_TIME=90
MAX_ATTEMPTS=10

for t in $(seq $trial); do
for i in $(seq 0 $((${#var[@]} - 1))); do
    indep=${var[${i}]}
    keys=${total_keys[${i}]}
    keys_per_client_first=$((keys / num_nodes_per_dc))
    keys_per_client=$((keys / nservers))
    cli_output_dir="/local/${dynamic_dir}/${exp_uid}/cops2/trial${t}"
    populate_attempts=0
    
    case $exp in
    size)
        value_size=$indep
        ;;
    columns_per_read)
        cols_per_key_read=$indep
        ;;
    colums_per_write)
        cols_per_key_write=$indep
        ;;
    keys_per_read)
        keys_per_read=$indep
        ;;
    keys_per_write)
        keys_per_write=$indep
        ;;
    write_frac)
        write_frac=$indep
        ;;
    write_trans_frac)
        write_trans_frac=$indep
        ;;
    esac
    
    data_file_name=${keys}_${value_size}_${cols_per_key_read}_${cols_per_key_write}_${keys_per_read}_${keys_per_write}_${write_frac}_${write_trans_frac}_${run_time}+${indep}+data
    
    cd ${cops_dir}
    sleep 1
    while [ 1 ]; do
        ./kodiak_dc_launcher.bash ${dcl_config_full}
        return_value=$?
        if [ $return_value -eq 0 ]; then
            break
        fi
    done
    
    set_server_keyspace
    
    while [ 1 ]; do
        (sleep $KILLALL_SSH_TIME; killall ssh) & killall_ssh_pid=$!
        
        pop_pids=""
        for dc in $(seq 0 $((ndcs - 1))); do
            local_servers_csv=$(echo ${servers_by_dc_auto[$dc]} | sed 's/ /,/g')
            
            for cli_index in $(seq 0 $((num_clients_per_dc - 1))); do
                client=$(echo ${clients_by_dc_auto[$dc]} | sed 's/ /\n/g' | head -n $((cli_index+1)) | tail -n 1)
                
                # Write to All so the cluser is populated everywhere
                ssh $client -t -t -o StrictHostKeyChecking=no "\
                    sudo mkdir -p /local/${dynamic_dir}/${exp_uid}; \
                    sudo chown ${username} /local/${dynamic_dir}; \
                    sudo chown ${username} /local/${dynamic_dir}/${exp_uid};"
                ssh $client -o StrictHostKeyChecking=no "\
                    $stress_killer; sleep 1; \
                    cd ${cops_dir}/tools/stress; \
                    bin/stress --nodes=$first_dc_servers_csv --columns=$cols_per_key_read --column-size=$value_size \
                    --operation=${insert_cmd} --consistency-level=LOCAL_QUORUM --replication-strategy=NetworkTopologyStrategy \
                    --strategy-properties=$strategy_properties --num-different-keys=$keys_per_client_first \
                    --num-keys=$keys_per_client_first --stress-index=$cli_index --stress-count=$num_nodes_per_dc \
                     > >(tee /local/${dynamic_dir}/${exp_uid}/populate${t}.out) \
                    2> >(tee /local/${dynamic_dir}/${exp_uid}/populate${t}.err) \
                    " 2>&1 | awk '{ print "'$client': "$0 }' & pop_pid=$!
                pop_pids="$pop_pids $pop_pid"
            done
        done
    
        # Wait for Clients to Finish
        for pop_pid in $pop_pids; do
            echo "Waiting on $pop_pid"
            wait $pop_pid
        done
        
        sleep 1
        
        kill $killall_ssh_pid
        killed_killall=$?
        if [ $killed_killall == "0" ]; then
            break;
        fi
        
        ((populate_attempts++))
        if [[ $populate_attempts -ge $MAX_ATTEMPTS ]]; then
            echo -e "\n\n \e[01;31m Could not populate the cluster after $MAX_ATTEMPTS attempts \e[0m \n\n"
            exit;
        fi
        echo -e "\e[01;31m Failed populating $populate_attempts times, trying again (out of $MAX_ATTEMPTS) \e[0m"
    done
    
    for dc in $(seq 0 $((ndcs - 1))); do
        local_servers_csv=$(echo ${servers_by_dc_auto[$dc]} | sed 's/ /,/g')
        
        for cli_index in $(seq 0 $((num_nodes_per_dc - 1))); do
            client=$(echo ${clients_by_dc_auto[$dc]} | sed 's/ /\n/g' | head -n $((cli_index+1)) | tail -n 1)
            
            ssh $client -t -t -o StrictHostKeyChecking=no "\
                sudo mkdir -p $cli_output_dir; \
                sudo chown ${username} $cli_output_dir;"
            ssh $client -o StrictHostKeyChecking=no "\
                cd ${cops_dir}/tools/stress; \
                ((bin/stress \
                --progress-interval=1 --nodes=$local_servers_csv --operation=DYNAMIC --consistency-level=LOCAL_QUORUM \
                --replication-strategy=NetworkTopologyStrategy --strategy-properties=$strategy_properties \
                --num-different-keys=$keys --stress-index=$cli_index --stress-count=$num_nodes_per_dc \
                --num-keys=20000000 --column-size=$value_size --columns-per-key-read=$cols_per_key_read \
                --columns-per-key-write=$cols_per_key_write --keys-per-read=$keys_per_read \
                --keys-per-write=$keys_per_write --write-fraction=$write_frac --write-transaction-fraction=$write_trans_frac \
                --threads=1 \
                 > >(tee ${cli_output_dir}/${data_file_name}) \
                2> ${cli_output_dir}/${data_file_name}.stderr) &); \
                sleep $((run_time + 10)); ${cops_dir}/kill_stress_kodiak.bash" \
                2>&1 | awk '{ print "'$client': "$0 }' &
        done
    done
    
    # Wait for Clients to Finish
    wait
    
    # Kill in Parallel
    set -m # need monitor mode to fg processes
    for ip in ${servers_by_dc_auto[@]}; do
        echo $ip
        ssh -t -t -o StrictHostKeyChecking=no $ip "cops2/kill_all_cassandra.bash" &
    done

    for ip in ${servers_by_dc_auto[@]}; do
        echo $ip
        fg
    done
    
    gather_results
    
done; done; exit;
EOF

# Data Gathering and Post Processing
Gather Data from Servers and Clients
    
Post Process the Data into Meaningful Chunks
    
Generate Graphs from the Data Chuncks

In [ ]:
server << 'EOF'

cd /users/$USER/cops2/experiments/

source setup.conf
output_dir=/users/${username}/cops2/experiments/"dynamic_"${exp}/latest

./dynamic_postprocess_full.b ash . ${output_dir} ${run_time} ${trim} shuffle; exit;
EOF